# Introduction to ForneyLab

ForneyLab is a toolbox for deriving solutions to probabilistic inference problems by message passing on Forney-style factor graphs. A Forney-style factor graph (FFG) is a graphical representation of a factorized probabilistic model. Message passing solutions to inference tasks can be automatically derived from the probabilistic model specification. ForneyLab yields custom message passing algorithms (MPA) as Julia code. In other words, ForneyLab is a Julia program that automatically builds inference programs that are specified as Julia programs.

These demos assume that the user is familiar with the FFG formalism. We recommend the following introductions: 

1. H.-A. Loeliger, J. Dauwels, J. Hu, S. Korl, Li Ping, and F. Kschischang,
[The factor graph approach to model-based signal processing](https://people.ee.ethz.ch/~papers/docu/aloe-jdau-juhu-skor-2007-1.pdf), Proceedings of the IEEE, vol. 95, no. 6, pp. 1295-1322, June 2007. 
2. Korl, Sascha, [A factor graph approach to signal modelling, system identification and filtering](https://www.research-collection.ethz.ch/handle/20.500.11850/82737), Series in signal and information processing
Doctoral Thesis, 2005

We designed ForneyLab to be practical, while retaining maximal flexibility. The inherent modularity of the FFG framework allowed us to make ForneyLab extensible at all levels (nodes, update rules, algorithms, inference engines). Although we had performance in mind while developing ForneyLab, optimally efficient execution of the resulting inference programs (specified in Julia as message passing sequence) may still require custom work.  

The ForneyLab approach to solving inference problems consists of three phases:

1. **Model specification**. ForneyLab provides a simple meta-language to specifiy models.
2. **Message Passing Agorithm (MPA) Generation**. This task is automatically performed by ForneyLab.
3. **MPA Execution**. This is simply evaluating a Julia program.

Each of the demos will step through these phases in turn, showcasing the most important ForneyLab functionalities. For more detailed information we refer to the Julia help functionality (simply type `?` and the ForneyLab function you're interested in), or the source code itself.

# Variables, Constants and Random Variables

The central concept in ForneyLab is the `Variable` type. After including ForneyLab and indicating that we start a new `FactorGraph`, we can declare a `Variable` as follows.

In [1]:
using ForneyLab

# Declare a new graph
g = FactorGraph()

# Declare a variable
x = Variable(id=:x);

The `FactorGraph` function also registers the newly created factor graph as the currently active graph. Since any model definition statements implicitly refer to the active graph, variable `x` is added to factor graph `g`. 

We can also define constants through the `constant()` function. This function declares a new variable and clamps it to the specified value:

In [2]:
# Declare variables m and v and clamp them to the specified values
m = constant(0.0, id=:m)
v = constant(1.0, id=:v);

Next, we can specify a (possibly probabilistic) relation among a set of variables. For instance, let us create a Gaussian node that constrains the relation between `x`, `m` and `v` to $x \sim \mathcal{N}(m,v)$: 

In [3]:
# Construct the model
GaussianMeanVariance(x, m, v, id=:my_gaussian)

# Inspect the graph (you need to to have GraphViz installed for this visualization to work)
ForneyLab.draw(g)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 14940392718489538960 -->
 14940392718489538960 
 
 m 
 
<!-- 9838758543777565859 -->
 9838758543777565859 
 
 𝒩 
 my_gaussian 
 
<!-- 9838758543777565859--14940392718489538960 -->
 9838758543777565859--14940392718489538960 
 
 m 
 1 out 
 2 m 
 
<!-- 3816686274348065052 -->
 3816686274348065052 
 
 v 
 
<!-- 9838758543777565859--3816686274348065052 -->
 9838758543777565859--3816686274348065052 
 
 v 
 1 out 
 3 v 
 
<!-- 182853494909701465272 -->
 182853494909701465272 
 
<!-- 182853494909701465272--9838758543777565859 -->
 182853494909701465272--9838758543777565859 
 
 x 
 1 out

ForneyLab comes equipped with the `~` operator to define random variables. As we see below, this operator also allows us to define the parameters of a model inline. 

In [4]:
# Start a new graph
g2 = FactorGraph()

# Construct the model
@RV x ~ GaussianMeanVariance(constant(0.0, id=:m), constant(1.0, id=:v))

# Inspect the graph
ForneyLab.draw(g2)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 14823432690910950482 -->
 14823432690910950482 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 8138262415040953427 -->
 8138262415040953427 
 
 v 
 
<!-- 14823432690910950482--8138262415040953427 -->
 14823432690910950482--8138262415040953427 
 
 v 
 1 out 
 3 v 
 
<!-- 7581542541204009622 -->
 7581542541204009622 
 
 m 
 
<!-- 14823432690910950482--7581542541204009622 -->
 14823432690910950482--7581542541204009622 
 
 m 
 1 out 
 2 m 
 
<!-- 143390357993223030142 -->
 143390357993223030142 
 
<!-- 143390357993223030142--14823432690910950482 -->
 143390357993223030142--14823432690910950482 
 
 x 
 1 out

# Distributions and Messages

Note that this graph represents a probability distribution
\begin{align*}
    p(x; m, v) = \mathcal{N}(x \,|\, m, v)
\end{align*}

In ForneyLab, probability distributions are always represented with respect to a node type with fixed parameters. This graph for example corresponds to the distribution

In [5]:
ProbabilityDistribution(Univariate, GaussianMeanVariance, m=0.0, v=1.0)

𝒩(m=0.00, v=1.00)


Messages that flow over edges are simply probability distributions with a scaling factor, and can be constructed as

In [6]:
Message(Univariate, GaussianMeanVariance, m=0.0, v=1.0)

Message: 𝒩(m=0.00, v=1.00)


After this short introduction we are prepared to start the first demo, on forward-only state estimation. Happy coding!